<a href="https://colab.research.google.com/github/mjkellner/MEC_Final/blob/main/Diffusion_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
## take the json file output from Openpose and convert it to a text file with only the body pose data

import os
import json

# Set the input and output folder paths
input_folder = '/content/drive/MyDrive/Colab_Notebooks/MEC_Final/Videos/Driver_Front_Source/output_json'
output_folder = '/content/drive/MyDrive/Colab_Notebooks/MEC_Final/Videos/Driver_Front_Source/output_txt'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop over each JSON file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Load the JSON data from the file
        with open(os.path.join(input_folder, filename)) as f:
            data = json.load(f)
        #if data['people'][0]['pose_keypoints_2d'] == True:     
        #  data2 = data['people'][0]['pose_keypoints_2d']

        # Create the output file path and name
        output_filename = os.path.splitext(filename)[0] + '.txt'
        output_filepath = os.path.join(output_folder, output_filename)
        
        # Print the output file path for debugging
        #print(output_filepath)

        # Write the data to the output file
        with open(output_filepath, mode='w') as txt_file:
            for item in data2:
              #print(item)
              txt_file.write(str('{:.2e}'.format(item)) + '\n')

In [10]:
import os
path = '/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/golfswing/img/Tiger_Front_img'
extension = '.jpg'

for i, filename in enumerate(os.listdir(path)):
    if filename.endswith(extension):
        new_filename = 'Tiger_Front_00{}.jpg'.format(i)
        old_path = os.path.join(path, filename)
        new_path = os.path.join(path, new_filename)
        os.rename(old_path, new_path)



In [11]:
path = '/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/golfswing/pose/Tiger_Front_txt'
extension = '.txt'

for i, filename in enumerate(os.listdir(path)):
    if filename.endswith(extension):
        new_filename = 'Tiger_Front_00{}.txt'.format(i)
        old_path = os.path.join(path, filename)
        new_path = os.path.join(path, new_filename)
        os.rename(old_path, new_path)

In [ ]:
import os
import re

folder1 = '/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/golfswing/img/Tiger_Back_img'
folder2 = '/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/golfswing/img/Tiger_Front_img'
output_file = '/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/golfswing/training_pairs.txt'

existing_pairs = set()

if os.path.exists(output_file):
    with open(output_file, 'r') as f:
        for line in f:
            pair = tuple(line.strip().split(','))
            existing_pairs.add(pair)

files1 = sorted(os.listdir(folder1))
files2 = sorted(os.listdir(folder2))

with open(output_file, 'a') as f:
    for filename1 in files1:
        match1 = re.search(r'(\d+)', filename1)
        for filename2 in files2:
            match2 = re.search(r'(\d+)', filename2)
            if match1.group(1) == match2.group(1):
                pair1 = (filename1, filename2)
                pair2 = (filename2, filename1)
                if pair1 not in existing_pairs and pair2 not in existing_pairs:
                    f.write(f"{filename1},{filename2}\n")
                    f.write(f"{filename2}, {filename1}\n")
                    existing_pairs.add(pair1) 
                    existing_pairs.add(pair2)

In [41]:
from sklearn.model_selection import train_test_split

filename = "/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/golfswing/training_pairs.txt"

# Read the lines from the text file
with open(filename, "r") as f:
    lines = f.readlines()

# Split the data into a training set and a test set
train_lines, test_lines = train_test_split(lines, test_size=0.3)

# Write the training set to a new text file
with open("/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/golfswing/trainfile.txt", "w") as f:
    f.writelines(train_lines)

# Write the test set to a new text file
with open("/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/golfswing/testfile.txt", "w") as f:
    f.writelines(test_lines)

In [55]:
!pip install lmdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 20.2 MB/s eta 0:00:00


In [75]:
!python /content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/PIDM/data/prepare_data.py \
--root /content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/golfswing/ \
--out /content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset/ 


100% 198/198 [00:04<00:00, 47.38it/s]


In [ ]:
!cd /content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/PIDM
# 1. Create a conda virtual environment.
!pip install torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0
!pip install -r /content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/PIDM/requirements.txt


In [ ]:
!python -m torch.distributed.launch \
--nproc_per_node=8 \
--master_port 48949 /content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/PIDM/train.py \
--dataset_path "/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/dataset" 


In [ ]:
#@title Pose Control using PIDM
%cd "/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/PIDM"
from predict import Predictor
from IPython.display import Image 
from google.colab import files 
obj = Predictor()

### upload a source file
#Path = list(files.upload().keys())[0] # Please check "/content/pidm-demo/data/" for sample images for source file.
### OR enter the path 
Path = "/content/drive/MyDrive/Colab_Notebooks/MEC_Final/Videos/Driver_Front_Source/org_images/Tiger Driver Front_1_000000000002.jpg"

obj.predict_pose(image=Path, sample_algorithm='ddim', num_poses=2, nsteps=50)

Image('output.png')

/content/drive/MyDrive/Colab_Notebooks/MEC_Final/PIDM_Diffusion/PIDM


ModuleNotFoundError: ignored